[![Open In Colab](https://colab.research.google.com/github/jgalazka/ERCC_analysis/blob/main/notebooks/GLDS-235_LVR_ERCC.ipynb)

# GeneLab ERCC spike-in analysis notebook

This notebook contains an analysis of RNA-seq counts data generated from ERCC spike-ins.

Here GLDS-235 is analyzed.

## Setting up the notebook

In [ ]:
# import python packages
import pandas as pd
pd.set_option('mode.chained_assignment', None) # suppress chained indexing warnings
import numpy as np
from urllib.request import urlopen, quote, urlretrieve
from json import loads
from re import search
import zipfile
import seaborn as sns
from scipy.stats import linregress
import matplotlib.pyplot as plt

## GeneLab metadata from URL

Before you begin, visit https://genelab-data.ndc.nasa.gov and browse a GLDS accession # of interest for your analysis.<br>
ISA.zip metadata from URL contains sample ID and transcription profile for RNA-Seq text files used in this analysis. <br>
Function below allows the user to retrieve ISA.zip file without relying on exact version of the data as the versions change.  

In [ ]:
# Function to pull metadata zip from GeneLab
# Credit to Kirill Grigorev
GENELAB_ROOT = "https://genelab-data.ndc.nasa.gov"
GLDS_URL_PREFIX = GENELAB_ROOT + "/genelab/data/study/data/"
FILELISTINGS_URL_PREFIX = GENELAB_ROOT + "/genelab/data/study/filelistings/"
ISA_ZIP_REGEX = r'.*_metadata_.*[_-]ISA\.zip$'

def read_json(url):
    with urlopen(url) as response:
        return loads(response.read().decode())

def get_isa(accession):
    glds_json = read_json(GLDS_URL_PREFIX + accession)
    try:
        _id = glds_json[0]["_id"]
    except (AssertionError, TypeError, KeyError, IndexError):
        raise ValueError("Malformed JSON?")
    isa_entries = [
        entry for entry in read_json(FILELISTINGS_URL_PREFIX + _id)
        if search(ISA_ZIP_REGEX, entry["file_name"])
    ]
    if len(isa_entries) == 0:
        raise ValueError("Unexpected: no ISAs found")
    elif len(isa_entries) > 1:
        raise ValueError("Unexpected: multiple files match the ISA regex")
    else:
        entry = isa_entries[0]
        version = entry["version"]
        url = GENELAB_ROOT + entry["remote_url"] + "?version={}".format(version)
        alt_url = (
            GENELAB_ROOT + "/genelab/static/media/dataset/" +
            quote(entry["file_name"]) + "?version={}".format(version)
        )
        return entry["file_name"], version, url, alt_url

The same GLDS accession # contains an unnormalized counts text file used in this analysis. <br>
Function below allows the user to retrieve Unnormalized_Counts.csv without relying on exact version of the data as the versions change.  

In [ ]:
# Function to pull unnormalized counts from GeneLab
# Credit to Kirill Grigorev

RAW_COUNTS_REGEX = r'.*_rna_seq_Unnormalized_Counts.csv'

def get_rawcounts(accession):
    glds_json = read_json(GLDS_URL_PREFIX + accession)
    try:
        _id = glds_json[0]["_id"]
    except (AssertionError, TypeError, KeyError, IndexError):
        raise ValueError("Malformed JSON?")
    raw_counts_entries = [
        entry for entry in read_json(FILELISTINGS_URL_PREFIX + _id)
        if search(RAW_COUNTS_REGEX, entry["file_name"])
    ]
    if len(raw_counts_entries) == 0:
        raise ValueError("Unexpected: no Raw Counts found")
    elif len(raw_counts_entries) > 1:
        raise ValueError("Unexpected: multiple files match the Raw Counts regex")
    else:
        entry = raw_counts_entries[0]
        version = entry["version"]
        url = GENELAB_ROOT + entry["remote_url"] + "?version={}".format(version)
        alt_url = (
            GENELAB_ROOT + "/genelab/static/media/dataset/" +
            quote(entry["file_name"]) + "?version={}".format(version)
        )
        return entry["file_name"], version, url, alt_url

## Get and parse data and metadata
Get and unzip ISA.zip, counts, and ERCC data.

In [ ]:
accession = 'GLDS-235' # Change this as necessary
isaurl = get_isa(accession)[3]
filehandle, _ = urlretrieve(isaurl)
zip_file_object = zipfile.ZipFile(filehandle, 'r')
zip_file_object.namelist() # Print contents of zip file. Pick relevant one from list

There are datasets that have multiple assays (including microarray), so holding ISA file based on location in above cell block will not always work. <br>Txt files outputted above are indexed as 0, 1, 2, etc. Fill in below at the end bracket of the number .txt file that corresponds to a for assay_file and s for sample_file.

In [ ]:
sample_file = zip_file_object.namelist()[0]
file = zip_file_object.open(sample_file)
sample_table = pd.read_csv(zip_file_object.open(sample_file), sep='\t')

assay_file = zip_file_object.namelist()[2]
file = zip_file_object.open(assay_file)
assay_table = pd.read_csv(zip_file_object.open(assay_file), sep='\t')

In [ ]:
sample_table.head(n=3)

In [ ]:
assay_table.head(n=3)

In [ ]:
# Get raw counts table
raw_counts_file = get_rawcounts(accession)[3]
raw_counts_table = pd.read_csv(raw_counts_file, index_col=0)
raw_counts_table.index.rename('Gene_ID', inplace=True)
raw_counts_table.head(n=3)

In [ ]:
raw_counts_transcripts = raw_counts_table[raw_counts_table.index.str.contains('^ENSMUSG')]
raw_counts_transcripts = raw_counts_transcripts.sort_values(by=list(raw_counts_transcripts), ascending=False)
raw_counts_transcripts

In [ ]:
#raw_counts_transcripts.to_csv('raw_counts_transcripts_stats.csv')

In [ ]:
# Get ERCC counts
ercc_counts = raw_counts_table[raw_counts_table.index.str.contains('^ERCC-')] 
ercc_counts.reset_index(inplace=True)
ercc_counts = ercc_counts.rename(columns={'Gene_ID':'ERCC ID'})
ercc_counts = ercc_counts.sort_values(by=list(ercc_counts), ascending=False)
ercc_counts.head()

In [ ]:
# Get ERCC files
ercc_url = 'https://assets.thermofisher.com/TFS-Assets/LSG/manuals/cms_095046.txt'
filehandle, _ = urlretrieve(ercc_url)
ercc_table = pd.read_csv(filehandle, '\t')
ercc_table.head(n=3)

## Number of ERCC Detected 
Number of ERCC detected in each of the 4 (A, B, C and D) groups for the study. 

Filter counts > 0

In [ ]:
meltERCC = ercc_counts.melt(id_vars=['ERCC ID'])
meltERCC['log2 Count'] = meltERCC['value']+1
meltERCC['log2 Count'] = np.log2(meltERCC['log2 Count'])
meltERCC = meltERCC.rename(columns={'variable':'Sample Name', 'value':'Count'})
meltERCC.head(n=3)

In [ ]:
# Build Mix dictionary to link sample name to mix added using assay table
mix_dict = assay_table.filter(['Sample Name','Parameter Value[Spike-in Mix Number]', 
                       'Parameter Value[Read Depth,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C155320,NCIT]'])
mix_dict = mix_dict.rename(columns={'Parameter Value[Spike-in Mix Number]':'Mix',
                                    'Parameter Value[Read Depth,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C155320,NCIT]':
                                    'Total Reads'})
mix_dict.head(n=3)

In [ ]:
# Make combined ercc counts and assay table
merged_ercc = meltERCC.merge(mix_dict, on='Sample Name')
merged_ercc.head(n=3)

In [ ]:
# Read ERCC info including concentrations from merged_ercc table
groupA = ercc_table.loc[ercc_table['subgroup'] == 'A']['ERCC ID']
groupB = ercc_table.loc[ercc_table['subgroup'] == 'B']['ERCC ID']
groupC = ercc_table.loc[ercc_table['subgroup'] == 'C']['ERCC ID']
groupD = ercc_table.loc[ercc_table['subgroup'] == 'D']['ERCC ID']

# Make a dictionary for ERCC group
group_dict = dict(zip(ercc_table['ERCC ID'], ercc_table['subgroup']))

In [ ]:
# Calculate Count per million and log2 Count per million
merged_ercc['Count per million'] = merged_ercc['Count'] / (merged_ercc['Total Reads'] / 1000000.0)
merged_ercc['log2 Count per million'] = np.log2(merged_ercc['Count per million']+1)

# Add ERCC group
merged_ercc['ERCC group'] = merged_ercc['ERCC ID'].map(group_dict)
merged_ercc

Filter and calculate mean log2 Count per million of Mix1 and Mix2 in each of the 4 groups

In [ ]:
# Filter Mix1 log2 CPM and Mix2 log2 CPM in group A 
# Mean values of Mix1 log2 CPM and Mix2 log2 CPM
Adf = merged_ercc.loc[merged_ercc['ERCC group'] == 'A']

Amix1df = Adf.loc[Adf['Mix']=='Mix 1']
Amix1df['Mix1 log2 CPM'] = Amix1df[Amix1df['log2 Count per million'] > 0]['log2 Count per million'].dropna()
Amix1df = Amix1df.groupby('ERCC ID')['Mix1 log2 CPM'].agg(np.mean).rename('ave Mix1 log2 CPM')
Amix1df = Amix1df.to_frame()
Amix2df = Adf.loc[Adf['Mix']=='Mix 2']
Amix2df['Mix2 log2 CPM'] = Amix2df[Amix2df['log2 Count per million'] > 0]['log2 Count per million'].dropna()
Amix2df = Amix2df.groupby('ERCC ID')['Mix2 log2 CPM'].agg(np.mean).rename('ave Mix2 log2 CPM')
Amix2df = Amix2df.to_frame()

adf = Amix1df.merge(Amix2df, on='ERCC ID', suffixes=('', '_2'))
adf = adf.reset_index()
adf['ave Mix1 log2 CPM/ ave Mix2 log2 CPM'] = (adf['ave Mix1 log2 CPM'] / adf['ave Mix2 log2 CPM'])

In [ ]:
Bdf = merged_ercc.loc[merged_ercc['ERCC group'] == 'B']
Bmix1df = Bdf.loc[Bdf['Mix']=='Mix 1']
Bmix1df['Mix1 log2 CPM'] = Bmix1df[Bmix1df['log2 Count per million'] > 0]['log2 Count per million'].dropna()
Bmix1df = Bmix1df.groupby('ERCC ID')['Mix1 log2 CPM'].agg(np.mean).rename('ave Mix1 log2 CPM')
Bmix1df = Bmix1df.to_frame()
Bmix2df = Bdf.loc[Bdf['Mix']=='Mix 2']
Bmix2df['Mix2 log2 CPM'] = Bmix2df[Bmix2df['log2 Count per million'] > 0]['log2 Count per million'].dropna()
Bmix2df = Bmix2df.groupby('ERCC ID')['Mix2 log2 CPM'].agg(np.mean).rename('ave Mix2 log2 CPM')
Bmix2df = Bmix2df.to_frame()

bdf = Bmix1df.merge(Bmix2df, on='ERCC ID')
bdf = bdf.reset_index()
bdf['ave Mix1 log2 CPM/ ave Mix2 log2 CPM'] = (bdf['ave Mix1 log2 CPM'] / bdf['ave Mix2 log2 CPM'])

In [ ]:
Cdf = merged_ercc.loc[merged_ercc['ERCC group'] == 'C']
Cmix1df = Cdf.loc[Cdf['Mix']=='Mix 1']
Cmix1df['Mix1 log2 CPM'] = Cmix1df[Cmix1df['log2 Count per million'] > 0]['log2 Count per million'].dropna()
Cmix1df = Cmix1df.groupby('ERCC ID')['Mix1 log2 CPM'].agg(np.mean).rename('ave Mix1 log2 CPM')
Cmix1df = Cmix1df.to_frame()
Cmix2df = Cdf.loc[Cdf['Mix']=='Mix 2']
Cmix2df['Mix2 log2 CPM'] = Cmix2df[Cmix2df['log2 Count per million'] > 0]['log2 Count per million'].dropna()
Cmix2df = Cmix2df.groupby('ERCC ID')['Mix2 log2 CPM'].agg(np.mean).rename('ave Mix2 log2 CPM')
Cmix2df = Cmix2df.to_frame()

cdf = Cmix1df.merge(Cmix2df, on='ERCC ID')
cdf = cdf.reset_index()
cdf['ave Mix1 log2 CPM/ ave Mix2 log2 CPM'] = (cdf['ave Mix1 log2 CPM'] / cdf['ave Mix2 log2 CPM'])

In [ ]:
Ddf = merged_ercc.loc[merged_ercc['ERCC group'] == 'D']
Dmix1df = Ddf.loc[Ddf['Mix']=='Mix 1']
Dmix1df['Mix1 log2 CPM'] = Dmix1df[Dmix1df['log2 Count per million'] > 0]['log2 Count per million'].dropna()
Dmix1df = Dmix1df.groupby('ERCC ID')['Mix1 log2 CPM'].agg(np.mean).rename('ave Mix1 log2 CPM')
Dmix1df = Dmix1df.to_frame()
Dmix2df = Ddf.loc[Ddf['Mix']=='Mix 2']
Dmix2df['Mix2 log2 CPM'] = Dmix2df[Dmix2df['log2 Count per million'] > 0]['log2 Count per million'].dropna()
Dmix2df = Dmix2df.groupby('ERCC ID')['Mix2 log2 CPM'].agg(np.mean).rename('ave Mix2 log2 CPM')
Dmix2df = Dmix2df.to_frame()

ddf = Dmix1df.merge(Dmix2df, on='ERCC ID')
ddf = ddf.reset_index()
ddf['ave Mix1 log2 CPM/ ave Mix2 log2 CPM'] = (ddf['ave Mix1 log2 CPM'] / ddf['ave Mix2 log2 CPM'])

Barplots of number of ERCC detected in group A.

In [ ]:
a = sns.catplot(x="ERCC ID", y="log2 Count per million", order=groupA, hue="Mix",data=merged_ercc[merged_ercc['ERCC ID'].isin(groupA)], kind="box", col="ERCC group", height=5, aspect=1)
a.set_xticklabels(rotation=90)
plt.text(23,1.75,"Mix1/ Mix2 = 4")
#figApath = 'groupA.pdf'
#a.savefig(figApath, bbox_inches='tight')

a = sns.catplot(x="ERCC ID", y="ave Mix1 log2 CPM/ ave Mix2 log2 CPM", palette="rocket_r", data=adf, kind="bar", 
               height=5, aspect=1, linewidth=0.5)
a.set_xticklabels(rotation=90)
plt.title("ERCC Group A")
a.set(ylim=(0, 7))
#figApath = 'groupA1.pdf'
#a.savefig(figApath, bbox_inches='tight')
print('Number of ERCC detected in group A (out of 23) =', adf['ave Mix1 log2 CPM/ ave Mix2 log2 CPM'].count())

Barplots of number of ERCC detected in group B.

In [ ]:
b = sns.catplot(x="ERCC ID", y="log2 Count per million", order=groupB, hue="Mix", data=merged_ercc[merged_ercc['ERCC ID'].isin(groupB)], kind="box", col="ERCC group", height=5, aspect=1)
b.set_xticklabels(rotation=90)
plt.text(23,1.25,"Mix1/ Mix2 = 1")

b = sns.catplot(x="ERCC ID", y="ave Mix1 log2 CPM/ ave Mix2 log2 CPM", palette="rocket_r", data=bdf, kind="bar", 
               height=5, aspect=1, linewidth=0.5)
b.set_xticklabels(rotation=90)
plt.title("ERCC Group B")
b.set(ylim=(0, 5))
print('Number of ERCC detected in group B (out of 23) =', bdf['ave Mix1 log2 CPM/ ave Mix2 log2 CPM'].count())

Barplots of number of ERCC detected in group C.

In [ ]:
c = sns.catplot(x="ERCC ID", y="log2 Count per million", order=groupC, hue="Mix", data=merged_ercc[merged_ercc['ERCC ID'].isin(groupC)], kind="box", col="ERCC group", height=5, aspect=1)
c.set_xticklabels(rotation=90)
plt.text(23,1.4,"Mix1/ Mix2 = 0.67")

c = sns.catplot(x="ERCC ID", y="ave Mix1 log2 CPM/ ave Mix2 log2 CPM", palette="rocket_r", data=cdf, kind="bar", 
               height=5, aspect=1, linewidth=0.5)
c.set_xticklabels(rotation=90)
plt.title("ERCC Group C")
c.set(ylim=(0, 2.5))
print('Number of ERCC detected in group C (out of 23) =', cdf['ave Mix1 log2 CPM/ ave Mix2 log2 CPM'].count())

Barplots of number of ERCC detected in group D.

In [ ]:
d = sns.catplot(x="ERCC ID", y="log2 Count per million", order=groupD, hue="Mix", data=merged_ercc[merged_ercc['ERCC ID'].isin(groupD)], col="ERCC group", kind="box", height=5, aspect=1)
d.set_xticklabels(rotation=90)
plt.text(23,1.5,"Mix1/ Mix2 = 0.5")

d = sns.catplot(x="ERCC ID", y="ave Mix1 log2 CPM/ ave Mix2 log2 CPM", palette="rocket_r", data=ddf, kind="bar", 
               height=5, aspect=1, linewidth=0.5)
d.set_xticklabels(rotation=90)
plt.title("ERCC Group D")
d.set(ylim=(0, 2))
print('Number of ERCC detected in group D (out of 23) =', ddf['ave Mix1 log2 CPM/ ave Mix2 log2 CPM'].count())

## Single Sample ERCC Analysis
Because samples may be combined by users outside of their original study, it is useful to have ERCC metrics that can be calculated from single samples. These could include: limit of detection, dynamic range, R^2 from measured vs expected plot, etc.

In [ ]:
ercc_table.head(n=3)

In [ ]:
# Make a dictionary for ERCC concentrations for each mix
mix1_conc_dict = dict(zip(ercc_table['ERCC ID'], ercc_table['concentration in Mix 1 (attomoles/ul)']))
mix2_conc_dict = dict(zip(ercc_table['ERCC ID'], ercc_table['concentration in Mix 2 (attomoles/ul)']))

Sometimes Spike-in Mix Number is spelled Spike-in Mix number.  Check assay_table header if you run into an error in the cell below.

In [ ]:
pd.set_option('max_columns', 80)
assay_table.head(n=3)

In [ ]:
# get samples that use mix 1 and mix 2
mix1_samples = assay_table[assay_table['Parameter Value[Spike-in Mix Number]'] == 'Mix 1']['Sample Name']
mix2_samples = assay_table[assay_table['Parameter Value[Spike-in Mix Number]'] == 'Mix 2']['Sample Name']

In [ ]:
# Get ERCC counts
ercc_counts = raw_counts_table[raw_counts_table.index.str.contains('^ERCC-')] 
ercc_counts = ercc_counts.sort_values(by=list(ercc_counts), ascending=False)
ercc_counts.head()

In [ ]:
ercc_counts_mix_1 = ercc_counts[mix1_samples]
ercc_counts_mix_1['ERCC conc (attomoles/ul)'] = ercc_counts_mix_1.index.map(mix1_conc_dict)
ercc_counts_mix_1.head(n=3)

In [ ]:
ercc_counts_mix_2 = ercc_counts[mix2_samples]
ercc_counts_mix_2['ERCC conc (attomoles/ul)'] = ercc_counts_mix_2.index.map(mix2_conc_dict)
ercc_counts_mix_2.head(n=3)

Each sample, either Mix 1 or Mix 2, plotted using scatter plot with x = log2 ERCC conc (attomoles/ ul) and y = log2 Counts.     

In [ ]:
columns_mix_1 = ercc_counts_mix_1.columns
columns_mix_2 = ercc_counts_mix_2.columns
all_columns = columns_mix_1.to_list() + columns_mix_2.to_list()
total_columns = len(columns_mix_1) + len(columns_mix_2) - 2
side_size = np.int32(np.ceil(np.sqrt(total_columns)))# calculate grid side size. take sqrt of total plots and round up.
fig, axs = plt.subplots(side_size+2, side_size-2, figsize=(20,20), sharex='all', sharey='all'); #change figsize x,y labels if needed.
fig.tight_layout(pad=1, w_pad=2.5, h_pad=3.5)

counter = 0
for ax in axs.flat:
    
    if(counter < len(columns_mix_1)-1):
      ax.scatter(x=np.log2(ercc_counts_mix_1['ERCC conc (attomoles/ul)']), y=np.log2(ercc_counts_mix_1[all_columns[counter]]+1), s=7);
      ax.set_title(all_columns[counter][-45:], fontsize=7.4);
      ax.set_xlabel('log2 ERCC conc (attomoles/ ul)', fontsize=7.5);
      ax.set_ylabel('log2 Counts', fontsize=7.5);
      ax.tick_params(direction='in', axis='both', labelsize=8, labelleft=True, labelbottom=True);
      
    elif(counter >= len(columns_mix_1) and counter < total_columns):
      ax.scatter(x=np.log2(ercc_counts_mix_2['ERCC conc (attomoles/ul)']), y=np.log2(ercc_counts_mix_2[all_columns[counter]]+1), s=7);
      ax.set_title(all_columns[counter][-45:], fontsize=7.4);
      ax.set_xlabel('log2 ERCC conc (attomoles/ ul)', fontsize=7.5);
      ax.set_ylabel('log2 Counts', fontsize=7.5);
      ax.tick_params(direction='in', axis='both', labelsize=8, labelleft=True, labelbottom=True);
       
    else:
      pass

    counter = counter + 1

Filter counts > 0

In [ ]:
nonzero_counts_list_1 = []
for i in range(0, len(ercc_counts_mix_1.columns)-1):
  counts = ercc_counts_mix_1[columns_mix_1[i]]
  counts.index.rename('Gene_ID', inplace=True)
  countsdf = pd.DataFrame(counts)
  nonzero_counts = countsdf[ercc_counts_mix_1[columns_mix_1[i]] > 0.0]
  nonzero_counts['Conc'] = nonzero_counts.index.map(mix1_conc_dict)
  nonzero_counts.columns = ['Counts','Conc']
  nonzero_counts_sorted = nonzero_counts.sort_values('Conc')
  nonzero_counts_list_1.append(nonzero_counts_sorted)

nonzero_counts_list_2 = []
for i in range(0, len(ercc_counts_mix_2.columns)-1):
  counts = ercc_counts_mix_2[columns_mix_2[i]]
  counts.index.rename('Gene_ID', inplace=True)
  countsdf = pd.DataFrame(counts)
  nonzero_counts = countsdf[ercc_counts_mix_2[columns_mix_2[i]] > 0.0]
  nonzero_counts['Conc'] = nonzero_counts.index.map(mix2_conc_dict)
  nonzero_counts.columns = ['Counts','Conc']
  nonzero_counts_sorted = nonzero_counts.sort_values('Conc')
  nonzero_counts_list_2.append(nonzero_counts_sorted)
    

Each sample plotted using linear regression of scatter plot with x = log2 Conc and y = log2 Counts.  Return min, max, R^2 and dynamic range (max / min) values.  

In [ ]:
samples = []
mins = []
maxs = []
dyranges = []
rs = []

fig, axs = plt.subplots(side_size+1, side_size-1, figsize=(25,20), sharex='all', sharey='all');
fig.tight_layout(pad=1, w_pad=2.5, h_pad=3.5)

counter = 0
list2counter = 0
for ax in axs.flat:
    
    if(counter < len(columns_mix_1)-1):

      nonzero_counts = nonzero_counts_list_1[counter]
      xvalues = nonzero_counts['Conc']
      yvalues = nonzero_counts['Counts']

      sns.regplot(x=np.log2(xvalues), y=np.log2(yvalues), ax=ax);
      ax.set_title(all_columns[counter][-47:], fontsize=8);
      ax.set_xlabel('log2 Conc (attomoles/ul)', fontsize=8);
      ax.set_ylabel('log2 Counts', fontsize=8);
      ax.tick_params(direction='in', axis='both', labelsize=8, labelleft=True, labelbottom=True)
      samples.append(all_columns[counter])

      if(len(xvalues) == 0):
        mins.append('NaN')
        maxs.append('NaN')
        dyranges.append('NaN')
        rs.append('NaN')

    
      else:
        min = xvalues[0];
        mins.append(min)
        minimum = f'Min:{min:.1f}';
        max = xvalues[-1];
        maxs.append(max)
        maximum = f'Max:{max:.1f}';
        dynamic_range = max / min;
        dyranges.append(dynamic_range)
        dyn_str = f'Dyn:{dynamic_range:.1f}';

        ax.text(0.02, 0.98, minimum,
        verticalalignment='top', horizontalalignment='left',
        transform=ax.transAxes,
        color='black', fontsize=10);
      
        ax.text(0.02, 0.88, maximum,
        verticalalignment='top', horizontalalignment='left',
        transform=ax.transAxes,
        color='black', fontsize=10);
      
        ax.text(0.02, 0.78, dyn_str,verticalalignment='top',
                horizontalalignment='left',transform=ax.transAxes,
                color='black', fontsize=10);
      
        if(len(xvalues) == 1):
          rs.append('NaN')

        else:
          slope, intercept, r, p, se = linregress(np.log2(xvalues), y=np.log2(yvalues))
          r_str = f'R:{r:.2f}'
          rs.append(r)

          ax.text(0.02, 0.68, r_str, verticalalignment='top',
                  horizontalalignment='left',transform=ax.transAxes,
                  color='black', fontsize=10);
    
    elif(counter >= len(columns_mix_1) and counter < total_columns):
      
      nonzero_counts = nonzero_counts_list_2[list2counter]
      xvalues = nonzero_counts['Conc']
      yvalues = nonzero_counts['Counts']

      sns.regplot(x=np.log2(xvalues), y=np.log2(yvalues), ax=ax);
      ax.set_title(all_columns[counter][-47:], fontsize=8);
      ax.set_xlabel('log2 Conc (attomoles/ul)', fontsize=8);
      ax.set_ylabel('log2 Counts', fontsize=8);
      ax.tick_params(direction='in', axis='both', labelsize=8, labelleft=True, labelbottom=True);
      samples.append(all_columns[counter])


      if(len(xvalues) == 0):
        mins.append('NaN')
        maxs.append('NaN')
        dyranges.append('NaN')
        rs.append('NaN')
    
      else:
        min = xvalues[0];
        mins.append(min)
        minimum = f'Min:{min:.1f}';
        max = xvalues[-1];
        maxs.append(max)
        maximum = f'Max:{max:.1f}';
        dynamic_range = max / min;
        dyranges.append(dynamic_range)
        dyn_str = f'Dyn:{dynamic_range:.1f}';

        ax.text(0.02, 0.98, minimum,
        verticalalignment='top', horizontalalignment='left',
        transform=ax.transAxes,
        color='black', fontsize=10);
      
        ax.text(0.02, 0.88, maximum,
        verticalalignment='top', horizontalalignment='left',
        transform=ax.transAxes,
        color='black', fontsize=10);
      
        ax.text(0.02, 0.78, dyn_str,verticalalignment='top',
                horizontalalignment='left',transform=ax.transAxes,
                color='black', fontsize=10);
      
        if(len(xvalues) == 1):
          rs.append('NaN')
          
        else:
          slope, intercept, r, p, se = linregress(np.log2(xvalues), y=np.log2(yvalues));
          r_str = f'R:{r:.2f}';
          rs.append(r)

          ax.text(0.02, 0.68, r_str, verticalalignment='top',
                  horizontalalignment='left',transform=ax.transAxes,
                  color='black', fontsize=10);

      list2counter = list2counter + 1
    
    else:
      pass

    counter = counter + 1

In [ ]:
# Directory for saved files
%mkdir ERCC

Change to working GLDS# in the following files.

In [ ]:
# Remember to change file names to GLDS# analyzing
# Select files w/ suffix _mqc* prepare for MultiQC report 
stats = pd.DataFrame(list(zip(samples, mins, maxs, dyranges, rs)))
stats.columns = ['Samples', 'Min', 'Max', 'Dynamic range', 'R']
stats.to_csv('ERCC/ERCC_stats_GLDS-235.csv', index = False) 
stats.filter(items = ['Samples', 'Dynamic range']).to_csv('ERCC/ERCC_dynrange_GLDS-235_mqc.csv', index = False)
stats.filter(items = ['Samples', 'R']).to_csv('ERCC/ERCC_rsq_GLDS-235_mqc.csv', index = False)

## GeneLab Study Level ERCC 
This section shows an example of how the ERCC spike-ins can be used to assess the power of comparisons within a single study.

In certain studies ERCC Mix 1 and Mix 2 are distributed so that all samples in Group 1 (e.g. Spaceflight) have Mix 1 and all samples in Group 2 (e.g. Ground Control). As certain transcripts in Mix 1 and Mix 2 are present at a known ratio, we can determine how well these patterns are revealed in the dataset.

In [ ]:
combined = sample_table.merge(assay_table, on='Sample Name')
combined = combined.set_index(combined['Sample Name'])
pd.set_option('max_columns', None)
combined.head(n=3)

Other GLDS studies (ie. Spaceflight) have other factor values.  It's important to inspect the factor values in your analysis. 

In [ ]:
# Select Euthanasia Method and Carcass Preservation Method  
euthanasia = combined['Factor Value[Euthanasia Method]'].isin(['CO2', 'Ketamine/xylazine IP'])
ln2 = combined['Factor Value[Carcass Preservation Method]'] == 'Liquid Nitrogen'

In [ ]:
# Sometimes Number in [Spike-in Mix Number] is spelled 'number' and this could cause error in mismatch search 
fewsamples = combined[euthanasia].loc[:,['Factor Value[Euthanasia Method]','Parameter Value[Spike-in Mix Number]']]
fewsamples.index = fewsamples.index.str.replace('-','_')

In [ ]:
fewsamples.columns = ['Euthanasia','Mix']
fewsamples.to_csv('ERCC/fewsamples.csv')

In [ ]:
ercc_counts.columns = ercc_counts.columns.str.replace('-','_')
fewcounts = ercc_counts.loc[:,fewsamples.index]

In [ ]:
fewcounts.to_csv('ERCC/fewcounts.csv')

## Perform DESeq2 analysis of ERCC counts

Switch to R script to perform DE on ERCC unnormalized counts. Contrast (pair-wise) Mix 1 and Mix 2 comparison returns mean normalized counts, log2FC (fold change), p-value, adj. p-value.

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("DESeq2")

In [ ]:
%%R
library("DESeq2")

In [ ]:
%%R
cts <- as.matrix(read.csv('ERCC/fewcounts.csv',sep=",",row.names="Gene_ID"))
coldata <- read.csv('ERCC/fewsamples.csv', row.names=1)

In [ ]:
%%R
coldata$Euthanasia <- factor(coldata$Euthanasia)
coldata$Mix <- factor(coldata$Mix)

In [ ]:
%%R
all(rownames(coldata) == colnames(cts))

In [ ]:
%%R
dds <- DESeqDataSetFromMatrix(countData = cts,
                              colData = coldata,
                              design = ~ Mix)
dds

Not included here are standard deviation and average to Mix 1 and Mix 2 that are reported in conventional DESeq2. 

In [ ]:
%%R
dds <- DESeq(dds)
res <- results(dds, contrast=c("Mix","Mix 1","Mix 2"))
res

In [ ]:
%%R
write.csv(res, 'ERCC/ERCC_DESeq2.csv')

In [ ]:
%%R
normcounts = counts(dds, normalized=TRUE)
write.csv(normcounts, 'ERCC/ERCC_normcounts.csv')

## Analyze DESeq2 results

In [ ]:
import pandas as pd
from urllib.request import urlopen, quote, urlretrieve
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
deseq2out = pd.read_csv('ERCC/ERCC_DESeq2.csv', index_col=0)
#deseq2out.index = deseq2out.index.str.replace('_','-')
deseq2out.rename(columns ={'baseMean' : 'meanNormCounts'}, inplace = True)
deseq2out.head()

In [ ]:
# Get ERCC files
ercc_url = 'https://assets.thermofisher.com/TFS-Assets/LSG/manuals/cms_095046.txt'
filehandle, _ = urlretrieve(ercc_url)
ercc_table = pd.read_csv(filehandle, '\t', index_col='ERCC ID')
ercc_table.head(n=3)

In [ ]:
combined = deseq2out.merge(ercc_table, left_index=True, right_index=True)
combined.head()

In [ ]:
combined['cleaned_padj'] = combined['padj']
combined.loc[(combined.cleaned_padj < 0.00001),'cleaned_padj']=0.00001

combined['cleaned_pvalue'] = combined['pvalue']
combined.loc[(combined.cleaned_pvalue < 0.00001),'cleaned_pvalue']=0.00001

combined.head()

In [ ]:
# Remember to change file name to GLDS# analyzing
combined.filter(items = ['ERCC ID', 'meanNormCounts', 'cleaned_pvalue','cleaned_padj']).to_csv('ERCC/ERCC_lodr_GLDS-235_mqc.csv')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

sns.scatterplot(data=combined, x="meanNormCounts", y="cleaned_pvalue",
            hue="expected fold-change ratio",
                palette=['red','green','black','blue'], ax=ax)

sns.lineplot(data=combined, x="meanNormCounts", y="cleaned_pvalue",
            hue="expected fold-change ratio",
                palette=['red','green','black','blue'], ax=ax)

#g.set_xscale("log", base=2)
ax.set_xscale("linear");
ax.set_yscale("log");

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

sns.scatterplot(data=combined, x="meanNormCounts", y="cleaned_padj",
            hue="expected fold-change ratio",
                palette=['red','green','black','blue'], ax=ax)

sns.lineplot(data=combined, x="meanNormCounts", y="cleaned_padj",
            hue="expected fold-change ratio",
                palette=['red','green','black','blue'], ax=ax)

#g.set_xscale("log", base=2)
ax.set_xscale("linear");
ax.set_yscale("log");

Run MultiQC report on select *mqc files separately in command line. 